In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML # dùng để hiển thị 
%matplotlib inline 
import helper

#All the header files required for the code
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random

In [2]:
#Importing both the file using pandas 
movies = pd.read_csv('movies data/movies data.csv')
ratings = pd.read_csv('movies data/ratings data.csv')
users = pd.read_csv('movies data/users data.csv')
#Deleting unnecessary columns
movies = movies.drop('Unnamed: 0',axis = 1)
ratings = ratings.drop(['Unnamed: 0','Timestamp'],axis = 1)
users = users.drop(['Unnamed: 0'],axis = 1)

print('Movies:',len(movies))
print('Ratings:',len(ratings))
print('Users:',len(users))

Movies: 3883
Ratings: 1000209
Users: 6040


In [3]:
#Merging both the dataframes
data = pd.merge(ratings , users , how='outer', on='UserID')
data.head()

,UserID,MovieID,Rating,gender,age,occupation,zipcode,age_desc,occ_desc
0,1,1193,5,F,1,10,48067,Under 18,K-12 student
1,1,661,3,F,1,10,48067,Under 18,K-12 student
2,1,914,3,F,1,10,48067,Under 18,K-12 student
3,1,3408,4,F,1,10,48067,Under 18,K-12 student
4,1,2355,5,F,1,10,48067,Under 18,K-12 student


In [4]:
# Data Processing
# Converting Genres into different columns 
# Here we just create columns and put there initial value as 0

# x = data.Genres
# a = list()
# for i in x:
#     abc = i
#     a.append(abc.split('|'))
# a = pd.DataFrame(a)   
# b = a[0].unique()
# for i in b:
#     data[i] = 0
# data.head(2)

In [5]:
# we assign 1 to all the columns which are present in the Genres

# for i in b:
#     data.loc[data['Genres'].str.contains(i), i] = 1
# data.head(2)

In [6]:
# Now there is no use of genre 
# Since we have movie id so there is no need for movie names as well

data = data.drop(['gender','age','occupation','zipcode','age_desc','occ_desc'],axis =1)
data.head(50)

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [7]:
data.columns

Index(['UserID', 'MovieID', 'Rating'], dtype='object')

In [8]:
# Because of merging some null values are created
# KMeans không hoạt động nếu có NaN
data.isnull().sum()

UserID     0
MovieID    0
Rating     0
dtype: int64

In [9]:
#WE simply drop the null values coz the are not treatable
# data.dropna(inplace= True)
data.fillna(random.randint(0,0),inplace=True)

In [10]:
kmeanModel = KMeans(n_clusters=3).fit(data)
centroids = kmeanModel.cluster_centers_
print(centroids)

[[4.34030680e+03 2.82955423e+03 3.49456729e+00]
 [4.36819896e+03 9.03945122e+02 3.68850078e+00]
 [1.34937295e+03 1.87924818e+03 3.56747287e+00]]


In [11]:
# Creating an extra column in data for storing the cluster values
data['Cluster'] = kmeanModel.labels_
data['Cluster'].sample(n=10)

523991    1
978763    1
201743    2
355956    2
527543    0
50375     2
733613    0
617313    1
344052    2
150846    2
Name: Cluster, dtype: int32

In [12]:
# chỉ là Cluster rating -> cần tìm cluster user dựa trên rating
data['Cluster'].value_counts()

2    442770
1    282135
0    275304
Name: Cluster, dtype: int64

In [13]:
data.head()

,UserID,MovieID,Rating,Cluster
0,1,1193,5,2
1,1,661,3,2
2,1,914,3,2
3,1,3408,4,2
4,1,2355,5,2


In [14]:
# When we merge the dataframe for a single movie multiple rows were created so a single movie is allotted
# to many clusters so here we allot a single cluster to a movie 
# the Cluster which occurs maximum number of times is alloted to the movie  
u = []
def fiUsers(group):
    a = pd.DataFrame(group)
    b = pd.DataFrame(a['Cluster'].value_counts())
    d = a.index 
    c = [a['UserID'][d[0]],int(b.idxmax())]
    u.append(c)

In [15]:
data.groupby("UserID").apply(lambda x: fiUsers(x))

""


In [16]:
#CLUSTERING MOVIES
u = pd.DataFrame(u)
u.head()

,0,1
0,1,2
1,1,2
2,2,2
3,3,2
4,4,2


In [17]:
# I Dont know why always the column name shift according to its will :(
# Here just the column names are swapped
u.rename(columns = {0:'UserID',1:'Cluster'},inplace=True)
u.drop_duplicates(inplace=True)

In [18]:
u.head(100)

,UserID,Cluster
0,1,2
2,2,2
3,3,2
4,4,2
5,5,2
...,...,...
96,96,2
97,97,2
98,98,2
99,99,2


In [19]:
ratings = pd.read_csv('movies data/ratings data.csv')
new_data = pd.merge(u , ratings , how='outer', on='UserID')
new_data = new_data.drop('Unnamed: 0',axis = 1)

In [20]:
# These were the movies we deleted while merging the file  
new_data.isnull().sum()

UserID       0
Cluster      0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [21]:
# new_data = new_data.drop(['gender','age','occupation','zipcode','age_desc','occ_desc'],axis =1)
new_data.isnull().sum()

UserID       0
Cluster      0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [22]:
new_data.head(20)

,UserID,Cluster,MovieID,Rating,Timestamp
0,1,2,1193,5,978300760
1,1,2,661,3,978302109
2,1,2,914,3,978301968
3,1,2,3408,4,978300275
4,1,2,2355,5,978824291
5,1,2,1197,3,978302268
6,1,2,1287,5,978302039
7,1,2,2804,5,978300719
8,1,2,594,4,978302268
9,1,2,919,4,978301368


In [23]:
cluster0 = new_data[new_data['Cluster'] == 0]
cluster1 = new_data[new_data['Cluster'] == 1]
cluster2 = new_data[new_data['Cluster'] == 2]

In [24]:
# CLUSTER 0
cluster0_movie_ratings = pd.merge(cluster0 , movies[['MovieID', 'Title']], on='MovieID' )
cluster0_user_movie_ratings = pd.pivot_table(cluster0_movie_ratings, index='UserID', columns= 'Title', values='Rating')
cluster0_user_movie_ratings.head()

Title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
UserID,,,,,,,,,,,,,,,,,,,,,
2697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# CLUSTER 1
cluster1_movie_ratings = pd.merge(cluster1 , movies[['MovieID', 'Title']], on='MovieID' )
cluster1_user_movie_ratings = pd.pivot_table(cluster1_movie_ratings, index='UserID', columns= 'Title', values='Rating')
cluster1_user_movie_ratings.head()

Title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zeus and Roxanne (1997),eXistenZ (1999)
UserID,,,,,,,,,,,,,,,,,,,,,
2738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# CLUSTER 2
cluster2_movie_ratings = pd.merge(cluster2 , movies[['MovieID', 'Title']], on='MovieID' )
cluster2_user_movie_ratings1 = pd.pivot_table(cluster2_movie_ratings, index='UserID', columns= 'MovieID', values='Rating')

# TODO
cluster2_user_movie_ratings = cluster2_user_movie_ratings1.sample(20)
cluster2_user_movie_ratings.head(20)

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
863,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1433,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2731,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1846,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2375,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
722,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2701,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN


In [27]:
cluster2_user_movie_ratings.isnull().sum()
# cluster2_user_movie_ratings.fillna(random.randint(0,0),inplace=True)

MovieID
1       12
2       20
3       16
4       19
5       20
        ..
3948    17
3949    20
3950    20
3951    20
3952    20
Length: 3609, dtype: int64

In [28]:
print(cluster2_user_movie_ratings.head())
print(cluster2_user_movie_ratings.values)

MovieID  1     2     3     4     5     6     7     8     9     10    ...  \
UserID                                                               ...   
863       NaN   NaN   NaN   NaN   NaN   4.0   NaN   NaN   NaN   NaN  ...   
1433      3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2731      4.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
1846      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2375      NaN   NaN   5.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

MovieID  3943  3944  3945  3946  3947  3948  3949  3950  3951  3952  
UserID                                                               
863       NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1433      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2731      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
1846      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2375      NaN   NaN   NaN   NaN   NaN   NaN   N

In [29]:


# indices for vector
def specified_rating_indices(u):
    return list(map(tuple, np.where(np.isfinite(u))))

def mean(u):
    # may use specified_rating_indices but use more time
    specified_ratings = u[specified_rating_indices(u)]#u[np.isfinite(u)]
    m = sum(specified_ratings)/np.shape(specified_ratings)[0]
    return m

def all_user_mean_ratings(ratings_matrix):
    return np.array([mean(ratings_matrix[u, :]) for u in range(ratings_matrix.shape[0])])

def get_mean_centered_ratings_matrix(ratings_matrix):
    users_mean_rating = all_user_mean_ratings(ratings_matrix)
    mean_centered_ratings_matrix = ratings_matrix - np.reshape(users_mean_rating, [-1, 1])
    return mean_centered_ratings_matrix

# Ta có được ma trận sau khi đã trừ đi trung bình ratings như sau:
mean_centered_ratings_matrix = get_mean_centered_ratings_matrix(cluster2_user_movie_ratings.values)
print(mean_centered_ratings_matrix)

[[        nan         nan         nan ...         nan         nan
          nan]
 [-0.33707865         nan         nan ...         nan         nan
          nan]
 [-0.27142857         nan         nan ...         nan         nan
          nan]
 ...
 [        nan         nan -0.95454545 ...         nan         nan
          nan]
 [ 0.79710145         nan  0.79710145 ...         nan         nan
          nan]
 [ 0.6969697          nan         nan ...         nan         nan
          nan]]


<ipython-input-29-ed3e6ea176d9>:7: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  specified_ratings = u[specified_rating_indices(u)]#u[np.isfinite(u)]


In [30]:
ratings_matrix = cluster2_user_movie_ratings.values

def pearson(u, v):
    mean_u = mean(u)
    mean_v = mean(v)
    
    specified_rating_indices_u = set(specified_rating_indices(u)[0])
    specified_rating_indices_v = set(specified_rating_indices(v)[0])
    
    mutually_specified_ratings_indices = specified_rating_indices_u.intersection(specified_rating_indices_v)
    mutually_specified_ratings_indices = list(mutually_specified_ratings_indices)
    
    u_mutually = u[mutually_specified_ratings_indices]
    v_mutually = v[mutually_specified_ratings_indices]
      
    centralized_mutually_u = u_mutually - mean_u
    centralized_mutually_v = v_mutually - mean_v

    result = np.sum(np.multiply(centralized_mutually_u, centralized_mutually_v)) 
    result = result / (np.sqrt(np.sum(np.square(centralized_mutually_u))) * np.sqrt(np.sum(np.square(centralized_mutually_v))))

    return result

print(pearson(ratings_matrix[1, :], ratings_matrix[2, :]))

0.5423866916111575


<ipython-input-29-ed3e6ea176d9>:7: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  specified_ratings = u[specified_rating_indices(u)]#u[np.isfinite(u)]


In [31]:
def get_user_similarity_value_for(u_index, ratings_matrix):
    user_ratings = ratings_matrix[u_index, :]
    similarity_value = np.array([pearson(ratings_matrix[i, :], user_ratings) for i in range(ratings_matrix.shape[0])])
    return similarity_value

def get_user_similarity_matrix(ratings_matrix):
    similarity_matrix = []
    for u_index in range(ratings_matrix.shape[0]):
        similarity_value = get_user_similarity_value_for(u_index, ratings_matrix)
        similarity_matrix.append(similarity_value)
    return np.array(similarity_matrix)

user_similarity_matrix = get_user_similarity_matrix(ratings_matrix)
# print(user_similarity_matrix)

user_similarity_matrix_pd = pd.DataFrame(user_similarity_matrix)
user_similarity_matrix_pd.head(20)

<ipython-input-29-ed3e6ea176d9>:7: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  specified_ratings = u[specified_rating_indices(u)]#u[np.isfinite(u)]
<ipython-input-30-8e6285b4117b>:20: RuntimeWarning: invalid value encountered in double_scalars
  result = result / (np.sqrt(np.sum(np.square(centralized_mutually_u))) * np.sqrt(np.sum(np.square(centralized_mutually_v))))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.000000,-0.236641,0.406095,-0.626011,-0.378456,-1.000000,0.315765,-0.465148,0.370941,-0.023420,-0.552017,-0.214585,0.511157,0.110749,0.654110,0.468133,1.000000,-0.990394,0.074092,0.507343
1,-0.236641,1.000000,0.542387,NaN,-0.541873,-0.309811,0.019437,-0.106561,0.136962,0.397442,NaN,0.003503,-0.417166,-0.358258,0.200639,-0.665300,1.000000,1.000000,0.740646,0.727557
2,0.406095,0.542387,1.000000,0.113515,0.682209,-0.276060,0.323893,0.167308,-0.112823,0.344952,-0.654599,0.411109,-0.118092,0.013905,0.331423,0.114468,0.602973,-0.061645,0.112710,0.599029
3,-0.626011,NaN,0.113515,1.000000,0.273015,0.865865,0.356979,0.616617,0.049547,0.218321,-0.145745,0.796289,0.079839,0.480852,0.104499,-0.089416,-0.014767,0.779169,-0.591451,-0.272443
4,-0.378456,-0.541873,0.682209,0.273015,1.000000,NaN,-0.173846,-0.449945,-0.575493,-0.154881,-0.178296,-0.195042,0.515110,0.827202,0.210863,0.680393,-0.233198,-0.767157,-0.290931,0.151931
5,-1.000000,-0.309811,-0.276060,0.865865,NaN,1.000000,-0.601655,0.672146,0.691719,0.141103,0.182115,-0.082012,-0.048696,0.870115,NaN,0.040575,0.058722,NaN,0.152057,-0.967459
6,0.315765,0.019437,0.323893,0.356979,-0.173846,-0.601655,1.000000,0.461156,0.497843,0.564958,0.544516,0.940913,-0.051930,0.096011,0.207757,-0.543505,0.613434,0.567976,-0.492368,0.331957
7,-0.465148,-0.106561,0.167308,0.616617,-0.449945,0.672146,0.461156,1.000000,-0.323182,0.380924,-0.318061,0.066517,0.264778,-0.014128,0.400452,0.470455,0.618590,NaN,0.041657,0.374398
8,0.370941,0.136962,-0.112823,0.049547,-0.575493,0.691719,0.497843,-0.323182,1.000000,0.338665,1.000000,0.414061,0.083263,0.039328,-0.081397,-0.201751,-0.210521,NaN,-0.969874,NaN
9,-0.023420,0.397442,0.344952,0.218321,-0.154881,0.141103,0.564958,0.380924,0.338665,1.000000,-0.352764,0.346445,0.003439,-0.154096,0.008320,-0.260436,0.693739,-0.126359,0.279620,-0.019376


In [32]:
row, col = cluster2_user_movie_ratings.shape
n_users = row
n_movies = col
print ('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 20 | Number of movies = 3609


In [33]:
R = mean_centered_ratings_matrix
print(R)
Ratings_demeaned = R;
# 1 cách khử bias khác
# user_ratings_mean = np.mean(R, axis = 1)
# Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)
# print(Ratings_demeaned)



[[        nan         nan         nan ...         nan         nan
          nan]
 [-0.33707865         nan         nan ...         nan         nan
          nan]
 [-0.27142857         nan         nan ...         nan         nan
          nan]
 ...
 [        nan         nan -0.95454545 ...         nan         nan
          nan]
 [ 0.79710145         nan  0.79710145 ...         nan         nan
          nan]
 [ 0.6969697          nan         nan ...         nan         nan
          nan]]


In [34]:
sparsity = round(1.0 - len(cluster2_user_movie_ratings) / float(n_users * n_movies), 3)
print ('The sparsity level of MovieLens1M dataset is ' +  str(sparsity * 100) + '%')

The sparsity level of MovieLens1M dataset is 100.0%


In [35]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(Ratings_demeaned, k = 10)

In [36]:
sigma = np.diag(sigma)

In [37]:
# Ma trận bias
users_mean_rating_all = all_user_mean_ratings(cluster2_user_movie_ratings.values)
print(users_mean_rating_all)

[4.22580645 3.33707865 4.27142857 4.03278689 3.7037037  3.73076923
 3.63809524 3.90909091 3.21929825 3.43309859 3.54054054 3.88489209
 3.90441176 3.88212928 4.10526316 4.13855422 4.47058824 3.95454545
 3.20289855 4.3030303 ]


<ipython-input-29-ed3e6ea176d9>:7: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  specified_ratings = u[specified_rating_indices(u)]#u[np.isfinite(u)]


In [38]:
# all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + np.reshape(users_mean_rating_all, [-1, 1])

print(all_user_predicted_ratings)

[[4.22580645 4.22580645 4.22580645 ... 4.22580645 4.22580645 4.22580645]
 [3.33707865 3.33707865 3.33707865 ... 3.33707865 3.33707865 3.33707865]
 [4.27142857 4.27142857 4.27142857 ... 4.27142857 4.27142857 4.27142857]
 ...
 [3.95454545 3.95454545 3.95454545 ... 3.95454545 3.95454545 3.95454545]
 [3.20289855 3.20289855 3.20289855 ... 3.20289855 3.20289855 3.20289855]
 [4.3030303  4.3030303  4.3030303  ... 4.3030303  4.3030303  4.3030303 ]]


In [39]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = cluster2_user_movie_ratings.columns)
preds.sample(10)

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
2,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429,...,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429,4.271429
18,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899,...,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899,3.202899
13,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129,...,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129,3.882129
12,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412,...,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412,3.904412
10,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541,...,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541,3.540541
14,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263,...,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263,4.105263
5,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769,...,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769,3.730769
4,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704,...,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704,3.703704
7,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091,...,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091,3.909091
15,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554,...,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554,4.138554


In [40]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.UserID == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [41]:
# already_rated, predictions = recommend_movies(preds, 1310, movies, ratings, 20)
already_rated, predictions = recommend_movies(preds, 8, movies, ratings, 20)

User 8 has already rated 139 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [42]:
# Top 20 movies that User 1310 has rated 
already_rated.head(20)

,Unnamed: 0,UserID,MovieID,Rating,Timestamp,Title,Genres
48,602,8,163,5,978229246,Desperado (1995),Action|Romance|Thriller
84,638,8,1721,5,978230800,Titanic (1997),Drama|Romance
81,635,8,3107,5,978229246,Backdraft (1991),Action|Drama
79,633,8,253,5,978230943,Interview with the Vampire (1994),Drama|Horror
33,587,8,1265,5,978229524,Groundhog Day (1993),Comedy|Romance
78,632,8,1213,5,978229817,GoodFellas (1990),Crime|Drama
117,671,8,2396,5,978229524,Shakespeare in Love (1998),Comedy|Romance
37,591,8,2427,5,978229204,"Thin Red Line, The (1998)",Action|Drama|War
38,592,8,58,5,978230286,"Postino, Il (The Postman) (1994)",Drama|Romance
115,669,8,3267,5,978229204,"Mariachi, El (1992)",Action|Thriller


In [43]:
# Top 20 movies that User 1310 hopefully will enjoy
predictions

,MovieID,Title,Genres
0,2,Jumanji (1995),Adventure|Children's|Fantasy
2511,2687,Tarzan (1999),Animation|Children's
2498,2673,Eternity and a Day (Mia eoniotita ke mia mera ...,Drama
2499,2674,"Loss of Sexual Innocence, The (1999)",Drama
2500,2675,Twice Upon a Yesterday (1998),Comedy|Drama|Romance
2501,2676,Instinct (1999),Drama|Thriller
2502,2677,Buena Vista Social Club (1999),Documentary
2503,2678,Desert Blue (1999),Drama
2504,2679,Finding North (1999),Drama|Romance
2506,2681,Free Enterprise (1998),Comedy|Romance|Sci-Fi


In [44]:
def __pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        # Step 1: find all users who rated i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        # Step 2: 
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        # Step 3: find similarity btw the current user and others 
        # who already rated i
        sim = self.S[u, users_rated_i]
        # Step 4: find the k most similarity users
        a = np.argsort(sim)[-self.k:]
        # and the corresponding similarity levels
        nearest_s = sim[a]
        # How did each of 'near' users rated item i
        r = self.Ybar[i, users_rated_i[a]]
        if normalized:
            # add a small number, for instance, 1e-8, to avoid dividing by 0
            return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)

        return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8) + self.mu[u]

In [45]:
# Step 1: find all users who rated i
ids = np.where(cluster2_user_movie_ratings[:, 1] == 163)[0].astype(np.int32)

TypeError: '(slice(None, None, None), 1)' is an invalid key